我们介绍了 n 元语法模型，
其中单词 $xt$ 在时间步 t 的条件概率仅取决于前面 n−1 个单词。
 对于时间步$ t−(n−1) $之前的单词， 如果我们想将其可能产生的影响合并到 $xt $上， 需要增加 n ，然而模型参数的数量也会随之呈指数增长，
 因为词表 V 需要存储 |V|n 个数字， 因此与其将 $P(xt∣xt−1,…,xt−n+1)$ 模型化， 不如使用隐变量模型：


$$P(xt∣x_{t−1},…,x1)≈P(xt∣h_{t−1})$$

其中 $h_{t−1}$ 是隐状态（hidden state）， 也称为隐藏变量（hidden variable）， 它存储了到时间步 t−1 的序列信息。 通常，我们可以基于当前输入 $x_t$ 和先前隐状态 $h_{t−1}$  来计算时间步 t 处的任何时间的隐状态：


$$h_t=f(xt,h_{t−1})$$

对于 上面中的函数 f ，隐变量模型不是近似值。 毕竟 $h_t$ 是可以仅仅存储到目前为止观察到的所有数据， 然而这样的操作可能会使计算和存储的代价都变得昂贵。

回想一下，我们在 4节中 讨论过的具有隐藏单元的隐藏层。 值得注意的是，隐藏层和隐状态指的是两个截然不同的概念。 如上所述，隐藏层是在从输入到输出的路径上（以观测角度来理解）的隐藏的层， 而隐状态则是在给定步骤所做的任何事情（以技术角度来定义）的输入， 并且这些状态只能通过先前时间步的数据来计算。

循环神经网络（recurrent neural networks，RNNs） 是具有隐状态的神经网络。 在介绍循环神经网络模型之前， 我们首先回顾 4.1节中介绍的多层感知机模型。

### 8.4.1. 无隐状态的神经网络
![多层感知机](imgs/8_4_1_无隐状态的神经网络1.png)

### 8.4.2. 有隐状态的循环神经网络
有了隐状态后，情况完全不同了。
假设我们在时间步t有小批量输入$X_t ∈ R^{n*d}$
换言之，对于n个序列样本的小批量，
##### $X_t$的每一行对应了时间步t的隐藏变量
与多层感知机不同的是，我们在这里保存了前一个时间步的隐藏变量$H_{t-1}$
并引入了一个新的权重参数$W_{hh} ∈ R^{hh}$，来描述如何在当前时间步中使用前一个时间步的隐藏变量。
具体的说，当前时间步隐藏变量是由当前时间步的输入
和前一个时间步的隐藏变量一起计算得出的。
$$H_t = ϕ(X_t W_{xh} + H_{t-1} W_{hh} + b_h)$$

可以看出，与MLP相比，上式多出了一项$H_{t-1}W_{hh}$
从相邻时间步的隐藏变量 Ht 和  Ht−1 之间的关系可知， 这些变量捕获并保留了序列直到其当前时间步的历史信息， 就如当前时间步下神经网络的状态或记忆， 因此这样的隐藏变量被称为隐状态（hidden state）。

##### 由于在当前时间步中，隐状态使用的定义与前一个时间步中使用的定义相同，因此计算式循环的recurrent。
 于是基于循环计算的隐状态神经网络被命名为 循环神经网络（recurrent neural network）。
 ![有隐状态的循环神经网络](imgs/8_4_2_有隐状态的循环神经网络1.png)

下图将会展示循环神经网络在相邻三个时间步的计算逻辑。
在任意时间步t，隐状态的计算可以视为：
1. 拼接当前时间步t的输入$X_t$和千亿时间步t-1的隐状态$H_{t-1}$
2. 将拼接的结果送入到带有激活函数ϕ 的全连接层。全连接层的输出是当前时刻的隐状态$H_t$

![具有隐状态的循环神经网络](imgs/8_4_2_有隐状态的循环神经网络2.png)

我们刚才提到，隐状态中
 $X_tW_{xh}+H_{t−1}W_{hh}$ 的计算，
 相当于 $X_t$ 和 $H_{t−1}$ 的拼接 与 $W_{xh}$ 和 $W_{hh}$ 的拼接的矩阵乘法。 虽然这个性质可以通过数学证明， 但在下面我们使用一个简单的代码来说明一下。

In [1]:
import torch
from d2l import torch as d2l

In [2]:
X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 2.1777,  2.6647, -0.0815, -0.5026],
        [ 1.1475,  3.8128,  0.4291,  1.2773],
        [ 2.4763,  3.4037,  0.2251, -1.9670]])

In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 2.1777,  2.6647, -0.0815, -0.5026],
        [ 1.1475,  3.8128,  0.4291,  1.2773],
        [ 2.4763,  3.4037,  0.2251, -1.9670]])

### 8.4.3. 基于循环神经网络的字符级语言模型
回想一下 8.3节中的语言模型， 我们的目标是根据过去的和当前的词元预测下一个词元， 因此我们将原始序列移位一个词元作为标签。 Bengio等人首先提出使用神经网络进行语言建模。 接下来，我们看一下如何使用循环神经网络来构建语言模型。 设小批量大小为1，批量中的那个文本序列为“machine”。 为了简化后续部分的训练，我们考虑使用 字符级语言模型（character-level language model）， 将文本词元化为字符而不是单词。 下图演示了 如何通过基于字符级语言建模的循环神经网络， 使用当前的和先前的字符预测下一个字符。
![基于循环神经网络的字符级语言模型](imgs/8_4_3_基于循环神经网络的字符级语言模型1.png)

在训练过程中，我们对每个时间步的输出层的输出进行softmax操作。
然后利用交叉熵损失计算误差。
由于隐藏层中隐状态的循环计算，上图中第3个时间步的输出$O_3$由文本序列m、a、c确定。
由于训练数据中这个文本序列的下一个字符是‘h’，因此第3个时间步的损失将取决于下一个字符的概率分布，而下一个字符是基于特征序列“m”、“a”、“c”和这个时间步的标签“h”生成的。
在实践中，我们使用的批量大小为 n>1 ， 每个词元都由一个 d 维向量表示。 因此，在时间步 t 输入 $X_t$ 将是一个 n×d 矩阵

### 8.4.4. 困惑度（Perplexity）
最后，让我们讨论如何度量语言模型的质量， 这将在后续部分中用于评估基于循环神经网络的模型。 一个好的语言模型能够用高度准确的词元来预测我们接下来会看到什么。 考虑一下由不同的语言模型给出的对“It is raining …”（“…下雨了”）的续写：

+ “It is raining outside”（外面下雨了）
+ “It is raining banana tree”（香蕉树下雨了）
+ “It is raining piouw;kcj pwepoiut”（piouw;kcj pwepoiut下雨了）

就质量而言，例 1 显然是最合乎情理、在逻辑上最连贯的。 虽然这个模型可能没有很准确地反映出后续词的语义， 比如，“It is raining in San Francisco”（旧金山下雨了） 和“It is raining in winter”（冬天下雨了） 可能才是更完美的合理扩展， 但该模型已经能够捕捉到跟在后面的是哪类单词。 例 2 则要糟糕得多，因为其产生了一个无意义的续写。 尽管如此，至少该模型已经学会了如何拼写单词， 以及单词之间的某种程度的相关性。 最后，例 3 表明了训练不足的模型是无法正确地拟合数据的。

我们可以通过计算序列的似然概率来度量模型的质量。 然而这是一个难以理解、难以比较的数字。 毕竟，较短的序列比较长的序列更有可能出现， 因此评估模型产生托尔斯泰的巨著《战争与和平》的可能性 不可避免地会比产生圣埃克苏佩里的中篇小说《小王子》可能性要小得多。 而缺少的可能性值相当于平均数。

![困惑度](imgs/8_4_4_困惑度1.png)

困惑度的最好的理解是“下一个词元的实际选择数的调和平均数”。 我们看看一些案例：

+ 在最好的情况下，模型总是完美地估计标签词元的概率为1。 在这种情况下，模型的困惑度为1。

+ 在最坏的情况下，模型总是预测标签词元的概率为0。 在这种情况下，困惑度是正无穷大。

+ 在基线上，该模型的预测是词表的所有可用词元上的均匀分布。 在这种情况下，困惑度等于词表中唯一词元的数量。 事实上，如果我们在没有任何压缩的情况下存储序列， 这将是我们能做的最好的编码方式。 因此，这种方式提供了一个重要的上限， 而任何实际模型都必须超越这个上限。